# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **  Since the solution to this problem involves preemptively labeling students as "needs intervention" or "does not need intervention", this is a classic binary classification problem.  If we wanted to find the mathematical relationship between certain attributes and student performance (e.g. how student absences affect their grade for a given class), then we would use regression instead.  Although this data could be valuable in a larger context, it is outside the bounds of our task, which only requires that we identify students in danger of failing and label them for intervention. 

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score
from IPython.display import Image 
from sklearn.tree import export_graphviz
import pydotplus

# Read student data
student_data = pd.read_csv("student-data.csv")
print("Student data read successfully!")
print(student_data.columns)


Student data read successfully!
Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc',
       'Walc', 'health', 'absences', 'passed'],
      dtype='object')


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
# TODO: Calculate number of students
n_students = student_data.count()[0]
# TODO: Calculate number of features
n_features = len(student_data.columns) - 1

passed_series = student_data['passed'].value_counts()
# TODO: Calculate passing students
n_passed = passed_series[0]

# TODO: Calculate failing students
n_failed = passed_series[1]

# # TODO: Calculate graduation rate
grad_rate = (n_passed / n_students) * 100

# Print the results
print ("Total number of students: {}".format(n_students))
print ("Number of features: {}".format(n_features))
print ("Number of students who passed: {}".format(n_passed))
print ("Number of students who failed: {}".format(n_failed))
print ("Graduation rate of the class: {:.2f}%".format(grad_rate))

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print ("Feature columns:\n{}".format(feature_cols))
print ("\nTarget column: {}".format(target_col))

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]
y_all = y_all.replace(['yes', 'no'], [1, 0])
# Show the feature information by printing the first five rows
print ("\nFeature values:")
print (X_all.head())

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print("Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns)))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [5]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split
# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size=num_train, random_state=13)

# Show the results of the split
print ("Training set has {} samples.".format(X_train.shape[0]))
print ("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 300 samples.
Testing set has 95 samples.


/Users/michaelkane/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **  

### Model 1: Naive Bayesian Classifier
Naive Bayesian Classifiers (NBCs) are a favorite machine learning algorithm for Natural Language Processing (NLP).  This model uses Bayesian Statistical Inference to predict the probabilities of different outcomes--this can be used for classification or regression.  Naive Bayesian Classifiers are a favorite for tasks that involve language such as spam detection--they can be used by examining individual word frequencies in a document, or by generalizing phrases to N-grams.  The model is called "Naive" because it works on the assumption that each feature is independent of the other features.  One of the strengths of the model is that NBCs tend to hold up well in high-dimensional spaces. However, the model is weak when there is correlation/multicollinearity between features, since the model expects features to be independent of one another.  

This model is a good fit for this problem since we have no reason to expect that there are strong levels of collinearity among features, and the model would still hold up well if stakeholders decided to add more dimensionality to our data set by considering other aspects of student performance that are not currently included.  This model is also a good choice since there are multiple versions of NBCs tailored to different statistical distributions (Gaussian, Bernoulli, and Multinomial NBCs).  


### Model 2: K-Nearest Neighbors Classifier
K-Nearest Neighbors Classifiers (KNNs) are a great choice for this problem.  KNN's are most widely used in the tech world for things such as recommendation engines on e-commerce websites.  These classifiers are based around the idea of similarity.  KNNs calculate similarity based on distance measurements (where distance is the inverse of similarity) by plotting each point in a space with arbitrary dimensionality. We use the labels of closest points to the point we're trying to predict (the point's "neighbors") to come up with our classification.  

KNNs are an instance-based learning algorithm.  This means that the algorithm doesn't create a model, but instead just stores all instances in memory.  This means that training time is O(1), since all computation is deferred until predictions are needed.  Runtime complexity for predictions is O(knd), where k is the number of neighbors to find, n is the number of data points, and d is the dimensionality of the data.  This means that the size and dimensionality of the data can have a huge effect on the runtime of this algorithm.  Although long runtimes can be a problem as the size and/or dimensionality of our data set scales, another strength of this algorithm is that it is parallelizable, meaning that we can spread the computational cost across many cores/computers to amortize our runtime.  Another weakness worth mentioning is that we don't intially know the ideal value for K, so we'll have to tune parameters in oder to discover this.  

This model is a good fit for this problem because it will likely be effective with this level of dimensionality.  Other reasons include explainability of results, since calculating similarity is an intutiive idea that anybody can understand.  This lends itself well to explaining model results to stakeholders.  The ability to parallelize processing is also nice, since it can help us cut down on runtime if needed (although this is a problem that can clearly be solved with batch processing, which means that rather than using multiple computing or paying for cloud computational power, we could save the school district money by just running the computation at times when people aren't working).  



### Model 3:  Decision Tree
Decision Tree Classifiers (DTCs) are a classic choice for this problem because of their long history of use in things such as risk modeling in finance.  This model is fairly robust regardless of the level of data preparation--we do not need to worry about things like normalizing the data.  Another strength of this model is that a visualization of the Decision Tree can easily be created to allow us to examine how the model arrived at a classification.  This model has some weaknesses as well-- DTCs are notorious for overfitting to the training data, creating complex decision boundaries that do not generalize well.  The model also does not perform well on non-orthogonal data, since decision boundaries are boolean (on a graph, they can only be vertical or horizontal).  Additional weaknesses include problems include creating biased trees when presented with data containing class imbalances.  

This is a model worth exploring for this problem because we are essentially modeling student risk, and this is a tried and true model for that sort of problem.  This is also a great choice because this is a "white-box" model with easily explainable results, which could conceivably be very important when explaining predictions to parents and other stakeholders in the school.  


### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [6]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print ("Trained model in {:.4f} seconds".format(end - start))

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print ("Made predictions in {:.4f} seconds.".format(end - start))
    return f1_score(target.values, y_pred)


def train_predict(clf, X_train, y_train, X_test, y_test):
    print("-"*50)
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print ("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print ("F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train)))
    print ("F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test)))
    print("-"*50)

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [7]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

# TODO: Initialize the three models
clf_A = GaussianNB()
clf_B = KNeighborsClassifier()
clf_C = DecisionTreeClassifier(random_state=13)

# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train
y_train_300 = y_train


# TODO: Execute the 'train_predict' function for each classifier and each training set size
#train_predict(clf_A), X_train, y_train, X_test, y_test)

In [8]:
# Train GaussianNB
# Train 100
train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)

# Train 200
train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)

# Train 300
train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)


--------------------------------------------------
Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0028 seconds
Made predictions in 0.0015 seconds.
F1 score for training set: 0.7377.
Made predictions in 0.0009 seconds.
F1 score for test set: 0.7241.
--------------------------------------------------
--------------------------------------------------
Training a GaussianNB using a training set size of 200. . .
Trained model in 0.0011 seconds
Made predictions in 0.0012 seconds.
F1 score for training set: 0.7774.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7049.
--------------------------------------------------
--------------------------------------------------
Training a GaussianNB using a training set size of 300. . .
Trained model in 0.0027 seconds
Made predictions in 0.0011 seconds.
F1 score for training set: 0.8152.
Made predictions in 0.0006 seconds.
F1 score for test set: 0.7556.
--------------------------------------------------


In [9]:
# Train KNN Classifier

# Train 100
train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)

# Train 200
train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)

# Train 300
train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)

--------------------------------------------------
Training a KNeighborsClassifier using a training set size of 100. . .
Trained model in 0.0032 seconds
Made predictions in 0.0026 seconds.
F1 score for training set: 0.7945.
Made predictions in 0.0020 seconds.
F1 score for test set: 0.7801.
--------------------------------------------------
--------------------------------------------------
Training a KNeighborsClassifier using a training set size of 200. . .
Trained model in 0.0010 seconds
Made predictions in 0.0048 seconds.
F1 score for training set: 0.8328.
Made predictions in 0.0036 seconds.
F1 score for test set: 0.7943.
--------------------------------------------------
--------------------------------------------------
Training a KNeighborsClassifier using a training set size of 300. . .
Trained model in 0.0027 seconds
Made predictions in 0.0090 seconds.
F1 score for training set: 0.8519.
Made predictions in 0.0039 seconds.
F1 score for test set: 0.7692.
-------------------------

In [10]:
# Train Random Forest

# Train 100
train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)

# Train 200
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)

# Train 300
train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)

--------------------------------------------------
Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0033 seconds
Made predictions in 0.0011 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0005 seconds.
F1 score for test set: 0.7154.
--------------------------------------------------
--------------------------------------------------
Training a DecisionTreeClassifier using a training set size of 200. . .
Trained model in 0.0018 seconds
Made predictions in 0.0006 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.7273.
--------------------------------------------------
--------------------------------------------------
Training a DecisionTreeClassifier using a training set size of 300. . .
Trained model in 0.0025 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0005 seconds.
F1 score for test set: 0.6306.
-------------------

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Gaussian Naive Bayes**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |            0.0031 seconds             |            0.0005 seconds            |         0.7377         |        0.7241         |
| 200               |        0.0011 seconds         |            0.0016 seconds            |        0.7774          |        0.7049         |
| 300               |            0.0013 seconds             |          0.0006 seconds              |        0.8152          |     0.7556     |

** Classifer 2 - K-Nearest Neighbors**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |           0.0008 seconds              |           0.0023 seconds             |        0.7945          |         0.7801        |
| 200               |     0.0008 seconds            |              0.0026 seconds          |        0.8328          |        0.7943        |
| 300               |        0.0009 seconds                 |            0.0035 seconds            |        0.8519          |     0.7692     |

** Classifer 3 - Decision Tree**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |            0.0033 seconds             |           0.0005 seconds             |       1.0000           |         0.7154        |
| 200               |           0.0018 seconds               |            0.0003 seconds            |         1.0000         |        0.7273         |
| 300               |             0.0025 seconds            |          0.0005 seconds              |        1.0000          |        0.6306         |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **  Of the three models tested, I recommend we use a K-Nearest Neighbors Classifier going forward.  This model provided the highest F1 score in testing, has acceptable space/time complexity for the size of our task, and the logic behind it is intuitive enough that it could be explained to students and parents without the use of jargon or complex math.  The KNN Classifier's F1 scores for testing at all sizes of training data still came in higher than any score by any other model, regardless of the size of the training set for that model.  Put simply, a KNN classifier with only 100 training instances still outscored a Gaussian Naive Bayesian Classifier trained on 300 training instances.  This model also significantly outscored the Random Forest Classifier by an average margin of 6-8%.  This makes sense when we consider the discrepancy between training and testing scores for the Random Forest Classifier, which shows telltale signs of overfitting (scoring near perfect on the training set but scoring significantly lower on the testing sets).  

One thing worth noting about the KNN Classifier is that as we add more data to our system, our training time for the model will not change since computation is deferred until predictions are made.  However, each new data point in our system will increase our runtime a little bit, since there are now more neighbors to model.  If we decide to begin tracking more features about students (I do not recommend this, as there is no reason to assume that this will improve our predictions--if anything, it will only cost us more time/money while having a negligible impact on performance), then the runtime for predictions would also increase.  Although this could conceivably take a long time to run if we begin feeding massive datasets through the model, there are simple management solutions that can deal with this.  Since we are performing computations on longitudinal student data and don't need to make snap decisions on incoming data, we can batch process these jobs easily, setting this to run overnight when we wouldnt normally be using the computer anyways.  If runtime is long enough that the processing time is untenable, we can also solve this problem by parallelizing the computation across multiple computers, or in the cloud.

Finally, I recommend this algorithm because the intuition behind why it works can be explained to parents and students in 1 or 2 simple sentences. See the answer to question 4 for more details on this.  

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: ** The K-Nearest Neighbors Classifier represents the best model for this task, because it is intuitively easy to understand how the model makes classifications.  The classifications are based on the idea of similarity--it compares things about a student (prior performance, absences, age, etc) to all other students.  It looks at the students are most similar to that student, and uses what we know about those students to make a prediction about our student.  Similarity is judged by imagining everything as points on a graph--the more similar two students are, the closer they are on the graph.  If more of those students needed intervention than didn't, then it predicts that this student will likely need intervention as well.  The model is also simple enough that we can see what things about the student caused it to make it's decision, which is important if parent, student, or court ever asks for clarification.  Although there are much more complex Deep Learning Models that could conceivably make more accurate predictions, it is hard to know how these models arrive at their decisions. With K-Nearest Neighbors, we can point to specific sorts of similarity metrics that are easy for non-technical people to understand (e.g. "Your child was labeled as needing intervention because [x] of the [k] students with the most similar attendance records and academic performance failed, and we want to catch this early to ensure that this doesn't happen to your child.")

From an operational standpoint, this model is also easy to automate.  It would be very easy to write programs that feed new data through the model at scheduled intervals (for instance, halfway through each quarter).  This automation could include auto-generated reports to stakeholders in the school system, such as administrators, teachers, or even parents of children at risk of failing.  Although a large increase in data could make runtimes for model predictions take longer, this is easily solvable by setting the model to run at opportune times such as late at night or on weekends.  As mentioned in the previous question, if time is an issue, we can always reduce runtime by having multiple computers work together to make the predictions.  

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [11]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score
# TODO: Create the parameters list you wish to tune
parameters =  { "n_neighbors": [i for i in range (1, 11)],
                "weights": ["uniform", "distance"],
                "algorithm": ["auto", "ball_tree", "kd_tree", "brute"],
              }

# TODO: Initialize the classifier
clf = KNeighborsClassifier()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score)

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_
print(clf)
# Report the final F1 score for training and testing after parameter tuning
print("Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train)))
print("Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test)))

KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=7, p=2,
           weights='uniform')
Made predictions in 0.0030 seconds.
Tuned model has a training F1 score of 0.8349.
Made predictions in 0.0010 seconds.
Tuned model has a testing F1 score of 0.7801.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **  The final F1 scores for the model are 0.8349 for Training and 0.7801 for Testing.  Paramater tuning has given us a modest but undeniable improvement of ~1-1.5% over the untuned model.  Although this gain may seem small, any increase in F1 score is worthwhile when it doesn't cost us a significant increase in space or runtime complexity.  

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.